Se importan las bibliotecas:

1. torch: proporciona el núcleo para construir y entrenar redes neuronales en PyTorch.
2. torch.nn: contiene los módulos de construcción de redes neuronales.
3. torch.optim: incluye optimizadores para entrenar el modelo.
4. torchvision.datasets y transforms: para cargar y transformar datos (en este caso, el conjunto MNIST).
5. DataLoader: para gestionar los datos de entrenamiento y prueba en lotes.

In [3]:
# Importar bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

1. Preprocesamiento de los datos:
   * transforms.Compose aplica transformaciones secuenciales:
   * transforms.ToTensor(): convierte las imágenes a tensores.
   * transforms.Normalize((0.5,), (0.5,)): normaliza los datos a un rango entre -1 y 1.
2. Carga del conjunto de datos MNIST:
   * datasets.MNIST: descarga el conjunto de datos MNIST.
   * train=True y train=False definen los datos de entrenamiento y prueba.
3. DataLoader:
   * train_loader y test_loader gestionan los datos en lotes de tamaño 64.
   * shuffle=True en train_loader mezcla los datos en cada época.

In [4]:
# Preprocesamiento y carga del conjunto de datos MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 9.91M/9.91M [00:01<00:00, 9.39MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 599kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 5.00MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



1. Esta clase define la arquitectura de la CNN:

    * conv1: primera capa convolucional con 32 filtros de 3x3, que toma imágenes con 1 canal.
    * conv2: segunda capa convolucional con 64 filtros de 3x3.
    * pool: capa de Max Pooling de tamaño 2x2, reduce la dimensionalidad.
    * fc1: primera capa densa con 128 neuronas.
    * fc2: capa de salida con 10 neuronas (una para cada clase en MNIST, dígitos del 0 al 9).
    
2. En el método forward se define el flujo de datos:

    * Las imágenes pasan por conv1, se aplica la activación ReLU y Max Pooling.
    * Luego pasan por conv2, se aplica ReLU y Max Pooling.
    * Aplanamiento: x.view(-1, 64 * 7 * 7) convierte la salida de las capas convolucionales en un vector.
    * fc1 y fc2 procesan el vector a través de capas densas, obteniendo la salida final.

In [5]:
# Definir la arquitectura de la CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Aplanar la salida
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Se inicializan:

* Modelo: instancia de la clase CNN.
* Función de pérdida: CrossEntropyLoss, adecuada para clasificación multiclase.
* Optimizador: Adam ajusta los pesos del modelo usando una tasa de aprendizaje de 0.001.

In [6]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

1. Configuración del entrenamiento:
    * El modelo se entrena por 5 épocas.
    * running_loss acumula la pérdida en cada lote para calcular la pérdida promedio.
2. Entrenamiento en cada lote:
   * optimizer.zero_grad(): reinicia los gradientes antes de cada actualización.
   * Propagación hacia adelante: model(images) calcula las predicciones.
3. Cálculo de la pérdida: criterion(outputs, labels) evalúa qué tan lejos están las predicciones de las etiquetas.
   * Propagación hacia atrás: loss.backward() calcula gradientes.
   * Actualización de pesos: optimizer.step() ajusta los pesos del modelo.
   * Impresión de la pérdida promedio: muestra la pérdida promedio por época.

In [7]:
# Entrenamiento del modelo
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Época {epoch+1}, Pérdida de entrenamiento: {running_loss/len(train_loader)}")


Época 1, Pérdida de entrenamiento: 0.1600990141394423
Época 2, Pérdida de entrenamiento: 0.04564293741044014
Época 3, Pérdida de entrenamiento: 0.029993323549318728
Época 4, Pérdida de entrenamiento: 0.02342565172795094
Época 5, Pérdida de entrenamiento: 0.017415261241508516


Evaluación del modelo:

1. torch.no_grad(): desactiva el cálculo de gradientes, optimizando la evaluación.
2. Propagación hacia adelante en cada lote: model(images) obtiene las predicciones.
3. torch.max(outputs.data, 1): obtiene la clase predicha con mayor probabilidad.
4. Calculo de precisión: calcula cuántas predicciones fueron correctas y divide entre el total para obtener la precisión en el conjunto de prueba.

In [8]:
# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Precisión en el conjunto de prueba:", correct / total)

Precisión en el conjunto de prueba: 0.9914
